In [2]:
import pandas as pd
import requests

In [20]:
def extraer_datos_por_genero_y_anio(anio_inicial, anio_final, genero , key):
    pelis = {}
    anio_distinto = False

    print(f"--- Extrayendo datos ---")
    print(" ")

    for anio in range(anio_inicial, anio_final+1): 

        print(f"Iterando por el año {anio}:")
        
        for pag in range(1, 51): # Itera por un rango determinado de páginas, ya que por cada página sólo se puede obtener un máximo de 50 entradas
            if anio_distinto: # Condición para que detenga el bucle si el año de lanzamiento es distinto al solicitado (variable = True)
                break

            url = "https://moviesdatabase.p.rapidapi.com/titles"
            querystring = {"genre": genero, "startYear": anio, "endYear": anio+1, "sort" : "year.incr" , "page": pag} # Parámetros de la URL
            headers = {"X-RapidAPI-Key": key, "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"}
            response = requests.get(url, headers=headers, params=querystring)
            response = response.json()

            try: # Intenta acceder al diccionario
                response['results']

            except: # Si no puede acceder significa que la url ha devuelto otra cosa y que, probablemente, se haya llegado al máximo de solicituds permitidas por hora
                print(f"Se ha llegado al límite de solicitudes. Deteniendo la iteración")
                break # Si se ha llegado al límite de solicitudes, detiene el bucle para que este no dé error

            # Revisa si la página que ha devuelto la API coincide con la página solicitada (se ha detectado que en el caso de no encontrar una página, la API devuelve la 1 por defecto y eso provocaba repeticiones de datos). 
            if response.get('page') != str(pag):
                print(f"La página {pag} no existe. Deteniendo la iteración.")
                break # Si los datos no coinciden, detiene el bucle

            # Revisa si no encuentra más entradas y en caso afirmativo detiene el bucle
            if len(response['results']) == 0:
                print(f"Ya no hay más resultados. Deteniendo la iteración")
                break
            
            # Itera por cada película/serie/elemento dentro de la lista de resultados
            for pelicula in response['results']:
                release_year = pelicula.get('releaseYear') # Asigna el valor de releaseYear a una variable

                # Revisa si el valor de releaseYear no es un None (es decir, existe) y si es distinto al año solicitado. En caso de que así sea, detiene el bucle
                if release_year is not None and pelicula['releaseYear']['year'] != anio:
                    print(f"Se ha detectado un año diferente a {anio}. Deteniendo la iteración")
                    anio_distinto = True  # Asigna el valor True a la variable anio_distinto para que sirva como parada del primer bucle for
                    break

            # Añade los resultados encontrados si ninguna de las anteriores condiciones para detener el bucle se ha cumplido y continúa con la búsqueda
            pelis[(anio, pag)] = response['results']

        print(" ")

    print(f"--- Extracción de datos finalizada ---")
    
    return pelis

In [26]:
anio_inicial = 2000
anio_final = 2004
genero = "Drama"
key = "22f73ac8d3msh933da10d160edbcp19332ajsndd3d7103a397" # Cambiar por la key personal

datos_drama = extraer_datos_por_genero_y_anio(anio_inicial , anio_final , genero , key)

--- Extrayendo datos ---
 
Iterando por el año 2000:
 
Iterando por el año 2001:
 
Iterando por el año 2002:
 
Iterando por el año 2003:
 
Iterando por el año 2004:
 
--- Extracción de datos finalizada ---


In [45]:
anio_inicial = 2000
anio_final = 2004
genero = "Comedy"
key = "b38cd0a830msh149cd093d780ffcp1fa141jsnc85ccd4fb846" # Cambiar por la key personal

datos_comedia = extraer_datos_por_genero_y_anio(anio_inicial , anio_final , genero , key)

--- Extrayendo datos ---
 
Iterando por el año 2000:
La página 51 no existe. Deteniendo la iteración.
 
Iterando por el año 2001:
La página 51 no existe. Deteniendo la iteración.
 
Iterando por el año 2002:
La página 51 no existe. Deteniendo la iteración.
 
Iterando por el año 2003:
La página 51 no existe. Deteniendo la iteración.
 
Iterando por el año 2004:
La página 51 no existe. Deteniendo la iteración.
 
--- Extracción de datos finalizada ---


In [50]:
anio_inicial = 2000
anio_final = 2004
genero = "Action"
key = "22f73ac8d3msh933da10d160edbcp19332ajsndd3d7103a397" # Cambiar por la key personal

datos_accion = extraer_datos_por_genero_y_anio(anio_inicial , anio_final , genero , key)

--- Extrayendo datos ---
 
Iterando por el año 2000:
La página 51 no existe. Deteniendo la iteración.
 
Iterando por el año 2001:
La página 51 no existe. Deteniendo la iteración.
 
Iterando por el año 2002:
La página 51 no existe. Deteniendo la iteración.
 
Iterando por el año 2003:
La página 51 no existe. Deteniendo la iteración.
 
Iterando por el año 2004:
La página 51 no existe. Deteniendo la iteración.
 
--- Extracción de datos finalizada ---


In [27]:
def limpiar_datos(dict_json):
    
    peliculas = []
    
    # Itera por cada clave del diccionario, es decir: Va entrando en 2000, 1 > 2000 , 2 ... 
    for anio_pagina in dict_json:
        
        # Itera por cada uno de los valores que hay por página y año
        for elemento in dict_json[anio_pagina]:
            
            # Condición para tener en cuenta sólo las películas (movies) y los cortometrajes (shorts)
            if elemento['titleType']['id'] == 'movie' or elemento['titleType']['id'] == 'short': 
                
                if elemento['titleType']['id'] == 'movie':
                    
                    tipo = 'Pelicula'
                    
                elif elemento['titleType']['id'] == 'short':
                    
                    tipo = 'Cortometraje'
        
                nombre = elemento['titleText']['text'] # Agrega el título de la película o el cortometraje
                anio_estreno = elemento['releaseYear']['year'] # Agrega el año de estreno
                id_pelicula = elemento['id'] # Agrega el valor de la ID
                release_date = elemento.get('releaseDate') # Asigna el valor de releaseDate a una variable 
                
                if release_date is not None: # Revisa que el valor de release_date no sea un None
                    mes_estreno = elemento['releaseDate']['month']  # Si no es un None, agrega el mes de estreno
                    
                else:
                    mes_estreno = "-" # Si el valor de release_date es un None, agrega "-" ya que no tenemos el dato del mes
                
                pelicula_corto = (tipo , nombre , anio_estreno , mes_estreno , id_pelicula) # Crea una tupla con todos los elementos de la película o el cortometraje
                peliculas.append(pelicula_corto) # Añade la tupla a la lista de tuplas
    
    return peliculas

In [28]:
drama2000_2004 = limpiar_datos(datos_drama)

In [47]:
comedia2000_2004 = limpiar_datos(datos_comedia)

In [51]:
accion2000_2004 = limpiar_datos(datos_accion)

In [10]:
def guardar_csv (dict_json):
    
    df_dict_json = pd.DataFrame(dict_json)
    
    df_dict_json.to_csv(input("Indica el nombre del archivo. Recuerda poner .csv al final"))

In [29]:
guardar_csv(drama2000_2004)

In [48]:
guardar_csv(comedia2000_2004)

In [52]:
guardar_csv(accion2000_2004)